In [58]:
import requests
import re
import json
import io
import csv
import os
import pandas as pd
import datetime
from io import StringIO
from openai import AzureOpenAI
from openai import OpenAI
import numpy as np
from time import sleep
import pyspark

In [60]:
import sys

In [61]:
# Project variables
import constants
locals().update(constants.subdirs)

In [62]:
# Project functions
from functions import getPromptResultsfromAPI

In [63]:
## Variables to Change
notesFile_version=constants.noteIDs[0]
goldStandard=True
runID="run1o2"
if goldStandard:
    notesV=notesFile_version
    notesFile_version=notesFile_version+"_"+runID

In [64]:
# Prompt Variables
import prompts.prompt_v2o1
from prompts.prompt_v2o1 import instructions
from prompts.prompt_v2o1 import question_p1
from prompts.prompt_v2o1 import getDataFrame_fromResponse_pt1
locals().update(prompts.prompt_v2o1.modelVars)

In [69]:
# In file - raw notes 
if goldStandard:
    f_Notes=datRaw+ "NotesbyID\\IDfiltered_onco_genetics_"+notesV+"_edit.csv"
else:
    f_Notes=datRaw+ "NotesbyID\\IDfiltered_onco_genetics_"+notesFile_version+".csv"

In [70]:
# Out file
f_p1NotesTMP=tmpOutDir+ "responsesP1_notes"+notesFile_version+"_OUTPUT_"+gptModel+"_"+promptV+"_TMP.txt"

In [71]:
variables = {}
with open(constants.f_mykey) as f:
    for line in f:
        name, value = line.split("=")
        variables[name] = str(value)

my_key = variables["my_key"]

In [72]:
notesDF=pd.read_csv(f_Notes, header = 0, converters={'mrn':str})

In [73]:
## local output file, script continuously writes to output, if local, less likely to crash
## if it doesnt exist, write file header/ temporary out file
if not (os.path.isfile(f_p1NotesTMP)):
    with open(f_p1NotesTMP, 'w', newline='') as filep1:
        tsv_writer = csv.writer(filep1, delimiter='\t', lineterminator='\n')
        tsv_writer.writerow(p1Cols)

In [75]:
# Run only the notes note already ran
p1TMP_In=pd.read_table(f_p1NotesTMP, header = 0, converters={'mrn':str})
notesDF=notesDF[~notesDF.note_id.isin(p1TMP_In.note_id)].reset_index(drop=True) # Remove rows that are already complete

In [76]:
print(instructions)

You are a genetics counselor knowledgeable in genetics testing for several cancer types. You are reading {} that may contain results and other information about a patient's history of genetics testing. When asked about genetics testing history, you will provide responses in as few words as possible without any additional details or summary text. Your answers will be precise, and responses will not be assumed from context.


In [77]:
print(question_p1)

For the genetics note pasted below, print one line of tab-delimited text with the following fields:

 - F1: Answer the question, 'Was genetic testing ordered in this patient?' Only answer 'yes' or 'no'.
 - F2: List the laboratory (e.g. 'Invitae', 'Ambry', 'Myriad', 'Foundation') that the testing was ordered from. Answer 'NA' if no testing was ordered 
 - F3: List the name of the genetics panel performed by the laboratory in F2. (e.g. 'CancerNext Expanded', 'Multi-gene cancer risk panel', 'Multi-gene breast/ovarian cancer risk panel'). If no testing panel name was provided or genetics testing was not ordered, answer 'N/A'.
 - F4: If explicitly given, include a comma separated list of all gene names that were tested for in the testing laboratories gene panel, regardless of results or pathogenic variant discovery. Answer 'NA', if a full list of genes was not given
 - F5: List the date that genetic testing was performed or results were returned. Only include a single date, in the format mm

In [78]:
tries=3 # will try note 3x before moving onto new note
total_notes = len(notesDF)
print("Total notes to process: " + str(total_notes))

Total notes to process: 1


In [ ]:
p1Results = list()
p1_skip = list()
total_notes = len(notesDF)
for i in range (0,total_notes):
        prompt=question_p1.format(notesDF['note'].iloc[i])
        mrn=notesDF['mrn'].iloc[i]
        NoteID=notesDF['note_id'].iloc[i]
        print("Getting results for prompt #{}: NoteID ={}".format(i+1, NoteID))

        for j in range(tries):   # Try 3 times before it gives up --if formatting is incorrect
            try:
                prompt_out=getPromptResultsfromAPI(gpturl,tries,gptModel,instructions,prompt,my_key)
                respDF=getDataFrame_fromResponse_pt1(mrn,prompt_out,NoteID)
            
                ## Check Proper Formatting
                if not isinstance(respDF, pd.DataFrame):
                    raise ValueError
                
            except:
                if j < tries - 1:
                    print("Response formatted incorrectly, retrying ...")
                    continue
                else:
                    p1_skip.append(notesDF.iloc[i])
            break
        p1Results.append(respDF)
        respDF.to_csv(f_p1NotesTMP,mode='a',sep='\t',index=False,header=False)
print("Done")